In [1]:
import networkx as nx

import pandas as pd
from operator import itemgetter
import matplotlib.pyplot as plt
import collections
from community import community_louvain
from networkx.algorithms.community.centrality import girvan_newman
import itertools
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
G = nx.Graph() # for a directed graph use nx.DiGraph()
G.add_node(1)
G.add_nodes_from(range(2,9))  # add multiple nodes at once

# add edges 
G.add_edge(1,2)
edges = [(2,3), (1,3), (4,1), (4,5), (5,6), (5,7), (6,7), (7,8), (6,8)]
G.add_edges_from(edges)
G.nodes()

NodeView((1, 2, 3, 4, 5, 6, 7, 8))

### Helper function for plotting the degree distribution of a Graph

In [ ]:
# Helper function for plotting the degree distribution of a Graph
def plot_degree_distribution(G):
    degrees = {}
    for node in G.nodes():
        degree = G.degree(node)
        if degree not in degrees:
            degrees[degree] = 0
        degrees[degree] += 1
    sorted_degree = sorted(degrees.items())
    deg = [k for (k,v) in sorted_degree]
    cnt = [v for (k,v) in sorted_degree]
    fig, ax = plt.subplots()
    plt.bar(deg, cnt, width=0.80, color='b')
    plt.title("Degree Distribution")
    plt.ylabel("Frequency")
    plt.xlabel("Degree")
    ax.set_xticks([d+0.05 for d in deg])
    ax.set_xticklabels(deg)

### Helper function for printing various graph properties: shortest path Length, Longest shortest path = diameter, Sparsity, Global clustering coefficient aka Transitivity

In [ ]:
# Helper function for printing various graph properties
def describe_graph(G):
    print(nx.info(G))
    if nx.is_connected(G):
        print("Avg. Shortest Path Length: %.4f" %nx.average_shortest_path_length(G))
        print("Diameter: %.4f" %nx.diameter(G)) # Longest shortest path
    else:
        print("Graph is not connected")
        print("Diameter and Avg shortest path length are not defined!")
    print("Sparsity: %.4f" %nx.density(G))  # #edges/#edges-complete-graph
    # #closed-triplets(3*#triangles)/#all-triplets
    print("Global clustering coefficient aka Transitivity: %.4f" %nx.transitivity(G))

### Helper function for visualizing the graph

In [ ]:
# Helper function for visualizing the graph
def visualize_graph(G, with_labels=True, k=None, alpha=1.0, node_shape='o'):
    #nx.draw_spring(G, with_labels=with_labels, alpha = alpha)
    pos = nx.spring_layout(G, k=k)
    if with_labels:
        lab = nx.draw_networkx_labels(G, pos, labels=dict([(n, n) for n in G.nodes()]))
    ec = nx.draw_networkx_edges(G, pos, alpha=alpha)
    nc = nx.draw_networkx_nodes(G, pos, nodelist=G.nodes(), node_color='g', node_shape=node_shape)
    plt.axis('off')

### Different Graphs: Erdős–Rényi, Zachary's Karate Club Network

In [ ]:
n = 10  # 10 nodes
m = 20  # 20 edges

erG = nx.gnm_random_graph(n, m)
# Zachary's Karate Club Network
karateG = nx.karate_club_graph()
nx.draw_circular(karateG, with_labels=True,  node_color='g', alpha = 0.8)

### Quaker network from pandas

In [ ]:
quakerG =nx.from_pandas_edgelist(edges, 'Source', 'Target', edge_attr=None, create_using= nx.Graph())
describe_graph(quakerG)

### Add nodes attributes

In [ ]:
# add node attributes by passing dictionary of type name -> attribute
nx.set_node_attributes(quakerG, nodes['Role'].to_dict(), 'Role' )
nx.set_node_attributes(quakerG, nodes['Gender'].to_dict(), 'Gender' )
nx.set_node_attributes(quakerG, nodes['Birthdate'].to_dict(), 'Birthdate' )
nx.set_node_attributes(quakerG, nodes['Deathdate'].to_dict(), 'Deathdate' )
nx.set_node_attributes(quakerG, nodes['Quaker'].to_dict(), 'Quaker' )

### Connected Components = set of nodes that are connected

In [ ]:
print(nx.is_connected(quakerG))
comp = list(nx.connected_components(quakerG))
print('The graph contains', len(comp), 'connected components')

### Node that have maximum connections, number of these connections == a giant component.

In [ ]:
largest_comp = max(comp, key=len)
percentage_lcc = len(largest_comp)/quakerG.number_of_nodes() * 100
print('The largest component has', len(largest_comp), 'nodes', 'accounting for %.2f'% percentage_lcc, '% of the nodes')

### Shortest Paths

In [ ]:
fell_whitehead_path = nx.shortest_path(quakerG, source="Margaret Fell", target="George Whitehead")
print("Shortest path between Fell and Whitehead:", fell_whitehead_path)

### Diameter of a giant component

In [ ]:
# take the largest component and analyse its diameter = longest shortest-path
lcc_quakerG = quakerG.subgraph(largest_comp)
print("The diameter of the largest connected component is", nx.diameter(lcc_quakerG))
print("The avg shortest path length of the largest connected component is", nx.average_shortest_path_length(lcc_quakerG))

### Transitivity = global clustering coefficient = the ratio of all existing triangles (closed triples) over all possible triangles (open and closed triplets)

In [ ]:
nx.transitivity(quakerG)

### clustering coefficient = the ratio of the number of edges to the number of all possible edges among the neighbors of a node.

In [ ]:
print(nx.clustering(quakerG, ['Alexander Parker', 'John Crook']))

### Draw subgraph

In [ ]:
# Lets check by looking at the subgraphs induced by Alex and John
subgraph_Alex = quakerG.subgraph(['Alexander Parker']+list(quakerG.neighbors('Alexander Parker')))
subgraph_John = quakerG.subgraph(['John Crook']+list(quakerG.neighbors('John Crook')))
nx.draw_spring(subgraph_Alex, with_labels=True)
nx.draw_circular(subgraph_John, with_labels=True)

# the most important quarkers: defined by degree, betweeness centrality, Katz Centrality (the generalization over degree centrality) 

### Degree: the more people you know, the more important you are!

In [ ]:
degrees = dict(quakerG.degree(quakerG.nodes()))
sorted_degree = sorted(degrees.items(), key=itemgetter(1), reverse=True)

# And the top 5 most popular quakers are.. 
for quaker, degree in sorted_degree[:5]:
    print(quaker, 'who is', quakerG.nodes[quaker]['Role'], 'knows', degree, 'people')

### Betweeness centrality: the more shortest paths pass through a node, the more important it is!

In [ ]:
# Compute betweenness centrality
betweenness = nx.betweenness_centrality(quakerG)
# Assign the computed centrality values as a node-attribute in your network
nx.set_node_attributes(quakerG, betweenness, 'betweenness')
sorted_betweenness = sorted(betweenness.items(), key=itemgetter(1), reverse=True)

for quaker, bw in sorted_betweenness[:5]:
    print(quaker, 'who is', quakerG.nodes[quaker]['Role'], 'has betweeness: %.3f' %bw)

### Katz Centrality (the generalization over degree centrality). If you were to have a directed one, use separate metrics for indegree and outdegree.

In [ ]:
degrees = dict(quakerG.degree(quakerG.nodes()))

katz = nx.katz_centrality(quakerG)
nx.set_node_attributes(quakerG, katz, 'katz')
sorted_katz = sorted(katz.items(), key=itemgetter(1), reverse=True)

# And the top 5 most popular quakers are.. 
for quaker, katzc in sorted_katz[:5]:
    print(quaker, 'who is', quakerG.nodes[quaker]['Role'], 'has katz-centrality: %.3f' %katzc)

### The quaker communities

### How likely is it that two quakers who have the same attribute are linked?

In [ ]:
nx.attribute_assortativity_coefficient(quakerG, 'Gender')
nx.numeric_assortativity_coefficient(quakerG, 'Deathdate')

# Handling networks exercise

### Create all the sets of people in the same scene:

In [ ]:
import re

def get_gossip(scene_lines):
    in_scene_characters = set(scene_lines["Character"])
    # {Penny, Leonard, Sheldon, Howard}
    return in_scene_characters

in_same_scene = lines_filtered.groupby(["Season", "Episode", "Scene"]).apply(get_gossip).reset_index(drop=True)
in_same_scene.head()

### Familiarity graph

In [ ]:
import collections

# Example with Counter
pairs = []
for idx, values in in_same_scene.iteritems():
    characters = list(values)
    while len(characters)>0:
        current = characters.pop(0)
        for c in characters:
            pairs.append(tuple(sorted([current, c])))
        
common_scenes = collections.Counter(pairs)

familiarity_graph = nx.Graph()

for key in common_scenes:
    familiarity_graph.add_edge(key[0], key[1], weight=common_scenes[key])

edges = familiarity_graph.edges()
weights = [0.01*familiarity_graph[u][v]['weight'] for u,v in edges]

nx.draw_shell(familiarity_graph, with_labels=True, alpha = 0.6, node_size=2000, width=weights)

### Gossip graph

In [ ]:
import re

def get_gossip(scene_lines):
    gossip_mentions = []
    # Characters speaking in the current scene
    in_scene_characters = set(scene_lines["Character"])
    for idx, row in scene_lines.iterrows():
        # split where is not a letter
        line_words = re.split("[^a-zA-Z]+", row["Line"])
        # Token is in the list of characters and not in the current scene
        gossip = [c for c in line_words if c in recurrent_characters and c not in in_scene_characters]
        if len(gossip)>0:
            gossip_mentions.append([{"Character": row["Character"], "Mention": c} for c in gossip])
    # Example: [[{'Character': 'Penny', 'Mentions': 'Raj'}], ...]
    return gossip_mentions

gm = lines_filtered.groupby(["Season", "Episode", "Scene"]).apply(get_gossip).reset_index(drop=True)

all_mentions = []
for idx, values in gm.iteritems():
    for mentions in values:
        all_mentions += mentions
        
all_mentions = pd.DataFrame(all_mentions)
all_mentions.head()
node_weights = all_mentions.value_counts(["Character", "Mention"]).reset_index()
node_weights.columns = ["Character", "Mention", "weight"]
node_weights.head()

In [ ]:
gossip_graph = nx.DiGraph()

for idx, r in node_weights.iterrows():
    gossip_graph.add_edge(r["Character"], r["Mention"], weight=r["weight"])
    
weighted_degree = dict(gossip_graph.degree(weight='weight'))

edges = gossip_graph.edges()
weights = [0.005*gossip_graph[u][v]['weight'] for u,v in edges]

nx.draw_shell(gossip_graph, with_labels=True, alpha = 0.6, width=weights,
              node_size=[v * 3 for v in weighted_degree.values()])

### Who is the most mentioned person = the biggest degree on node

In [ ]:
import collections

weighted_degree = collections.Counter(dict(gossip_graph.degree(weight='weight')))
weighted_degree.most_common()[0]

### Every character in the show is familiar with everyone else through at most one intermediary = Shortest path

In [ ]:
nodes = list(familiarity_graph.nodes)

one_intermediary = True
for source_idx in range(0, len(familiarity_graph.nodes)):
    for destination_idx in range(source_idx+1, len(familiarity_graph.nodes)):
        shortest_path = nx.shortest_path_length(familiarity_graph, 
                                                source=nodes[source_idx], 
                                                target=nodes[destination_idx])
        if shortest_path>2:
            one_intermediary = False
            break

print("The claim of Sheldon is {}".format(one_intermediary))

### The character through whom the largest number of these indirect familiarities happen = betweeness centrality

In [ ]:
most_central_people = sorted(nx.betweenness_centrality(familiarity_graph).items(), key=lambda r: -r[1])
most_central_people[:5]

### Check completeness of graph

In [ ]:
complete = True
for c in gossip_graph.nodes:
    in_degree = gossip_graph.in_degree(c, weight=None)
    if in_degree < total_characters-1:
        complete = False
        break
        
# Alternative: use out_degree

print("The claim of Sheldon that every recurrent character gossips about all the others is {}".format(complete))

### If for every pair of recurrent characters, one of them has gossiped about the other if and only if they know each other = has_edge from another graph

In [ ]:
# For every gossip edge, check if familiarity edge exists

for ge in gossip_graph.edges:
    source = ge[0]
    destination = ge[1]
    if not familiarity_graph.has_edge(source, destination):
        print("{} speaks about {} without sharing a scene".format(source, destination))